In [32]:
import sys
import os
import pathlib
sys.path.insert(0, '..')

import numpy as np
import dataclasses
import matplotlib.pyplot as plt
from IPython.display import Image

import control
import f16
import test_f16
import casadi as ca

np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=200)

In [33]:
@dataclasses.dataclass
class StateReduced(f16.CasadiDataClass):
    VT: float = 0  # true velocity, (ft/s)
    alpha: float = 0  # angle of attack, (rad)
    theta: float = 0  # B321 pitch angle, (rad)
    Q: float = 0  # body pitch rate, (rad/s)
    power: float = 0  # power, (0-1)
    beta: float = 0  # sideslip angle, (rad)
    phi: float = 0  # B321 roll angle, (rad)
    P: float = 0  # body roll rate, (rad/s)
    R: float = 0  # body yaw rate, (rad/s)

@dataclasses.dataclass
class StateDotReduced(f16.CasadiDataClass):
    VT_dot: float = 0  # true velocity derivative, (ft/s^2)
    alpha_dot: float = 0  # angle of attack rate, (rad/s)
    theta_dot: float = 0  # B321 pitch rate, (rad/s)
    Q_dot: float = 0  # body pitch accel, (rad/s^2)
    power_dot: float = 0  # power rate, (NA)
    beta_dot: float = 0  # sideslip rate, (rad/s)
    phi_dot: float = 0  # B321 roll rate, (rad/s)
    P_dot: float = 0  # body roll accel, (rad/s^2)
    R_dot: float = 0  # body yaw accel, (rad/s^2)

In [34]:
# trim
p0 = f16.Parameters(xcg=0.38)
x0 = f16.State(VT=502, alpha=0.03544, theta=0.03544)
u0 = f16.Control(thtl=0.1325, elv_deg=-0.0559)
tables = f16.build_tables()
x0.power = tables['tgear'](u0.thtl)
dx = f16.dynamics(x0, u0, p0, tables)
dx

StateDot(VT_dot=DM(-0.000991246), alpha_dot=DM(-8.68343e-07), beta_dot=DM(0), phi_dot=0.0, theta_dot=0.0, psi_dot=0.0, P_dot=DM(0), Q_dot=DM(2.29542e-06), R_dot=DM(0), V_N=502.0, V_E=0.0, alt_dot=0.0, power_dot=DM(0))

In [35]:
xR_sym = ca.MX.sym('xR', 9)
u_sym = ca.MX.sym('u_sym', 4)
xR = StateReduced.from_casadi(xR_sym)
x = f16.State(**xR.to_dict())
u = f16.Control.from_casadi(u_sym)
p = f16.Parameters()
tables = f16.build_tables()
dx = f16.dynamics(x, u, p, tables)
dxR = StateDotReduced(**{field.name: dx.to_dict()[field.name] for field in StateDotReduced.fields()})
xR0 = StateReduced(**{field.name: x0.to_dict()[field.name] for field in StateReduced.fields()})
f_A = ca.Function('A', [xR_sym, u_sym], [ca.jacobian(dxR.to_casadi(), xR_sym)])
A = np.array(f_A(
    np.array(xR0.to_tuple()),
    np.array(u0.to_tuple())))
A

array([[ -0.019,   9.378, -32.17 ,  -0.53 ,   0.402,   0.   ,   0.   ,   0.   ,   0.   ],
       [ -0.   ,  -1.019,   0.   ,   0.905,  -0.   ,   0.   ,  -0.   ,  -0.   ,  -0.   ],
       [  0.   ,   0.   ,   0.   ,   1.   ,   0.   ,   0.   ,  -0.   ,   0.   ,  -0.   ],
       [ -0.   ,   0.834,   0.   ,  -1.077,   0.   ,   0.   ,   0.   ,   0.   ,  -0.003],
       [  0.   ,   0.   ,   0.   ,   0.   ,  -1.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ,   0.   ,  -0.   ,  -0.322,   0.064,   0.035,  -0.992],
       [  0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   1.   ,   0.035],
       [  0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,  -3.681,   0.657],
       [  0.   ,   0.   ,   0.   ,   0.003,   0.   ,   0.   ,   0.   ,  -0.024,  -0.476]])

In [36]:
f_B = ca.Function('B', [xR_sym, u_sym], [ca.jacobian(dxR.to_casadi(), u_sym)])
B = np.array(f_B(
    np.array(xR0.to_tuple()),
    np.array(u0.to_tuple())))
B

array([[ 0.   ,  0.   ,  0.176,  0.   ],
       [ 0.   ,  0.   , -0.002,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.176,  0.   ],
       [64.94 ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.   ,  0.001],
       [ 0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.733,  0.   ,  0.132],
       [ 0.   , -0.032,  0.   , -0.062]])

In [40]:
ss_lon = control.ss(A[:5, :5], B[:5, :2], np.eye(5), np.zeros((5, 2)))
ss_lon

A = [[ -0.019   9.378 -32.17   -0.53    0.402]
 [ -0.     -1.019   0.      0.905  -0.   ]
 [  0.      0.      0.      1.      0.   ]
 [ -0.      0.834   0.     -1.077   0.   ]
 [  0.      0.      0.      0.     -1.   ]]

B = [[ 0.    0.  ]
 [ 0.    0.  ]
 [ 0.    0.  ]
 [ 0.    0.  ]
 [64.94  0.  ]]

C = [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

D = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

In [41]:
ss_lat = control.ss(A[5:, 5:], B[5:, 2:], np.eye(4), np.zeros((4, 2)))
ss_lat

A = [[-0.322  0.064  0.035 -0.992]
 [ 0.     0.     1.     0.035]
 [ 0.     0.    -3.681  0.657]
 [ 0.     0.    -0.024 -0.476]]

B = [[-0.     0.001]
 [ 0.     0.   ]
 [ 0.     0.132]
 [ 0.    -0.062]]

C = [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

D = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]